In [1]:
import pandas as pd
import numpy as np

directory of meta data ftp://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata

### input directory

In [2]:

basePickleDir='/data/cellardata/users/btsui/SRA/DUMP/'
accessionProjDir='/data/cellardata/users/btsui/SRA/DUMP/SRA_Accessions.tab'
accessionDir='/data/cellardata/users/btsui/SRA/DUMP/SRA_Run_Members.tab'

In [3]:
srsMergedS=pd.read_pickle(basePickleDir+'allSRS.pickle.gz')
srxMergedS=pd.read_pickle(basePickleDir+'allSRX.pickle.gz')

### output directory

In [4]:
sra_dump_csv_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.csv.gz'
sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'


In [5]:
projAccessDf=pd.DataFrame.from_csv(accessionProjDir,sep='\t')

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
accessionDf=pd.DataFrame.from_csv(accessionDir,sep='\t')

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### filter onto only human mouse, and zebra fish data

In [7]:
validScientificNameSrs=srsMergedS[srsMergedS.index.get_level_values(1)=='SCIENTIFIC_NAME']#[scientific_nameS.isin({'Homo sapiens','Mus musculus'})]
#rename the genomes to match the data

In [8]:
#validSrs=validScientificNameSrs.index.get_level_values(0)

In [9]:
srsToSpecies=pd.Series( index=validScientificNameSrs.index.get_level_values(0),data=validScientificNameSrs.values)

uniqueSrsToSpeciesS=srsToSpecies.groupby(srsToSpecies.index).first()

### filter onto to RNA-seq

In [10]:
srxMergedS.head(n=6)

ERX1269638  TITLE                Illumina HiSeq 1500 paired end sequencing
            LIBRARY_STRATEGY                                           EST
            LIBRARY_SOURCE                                         GENOMIC
            LIBRARY_SELECTION                                       RANDOM
            LIBRARY_LAYOUT                                          PAIRED
ERX1269639  TITLE                Illumina HiSeq 1500 paired end sequencing
dtype: object

In [11]:
layoutS=srxMergedS[srxMergedS.index.get_level_values(1)=='LIBRARY_LAYOUT'].groupby(level=0).first()

libraryS=srxMergedS[srxMergedS.index.get_level_values(1)=='LIBRARY_STRATEGY'].groupby(level=0).first()

In [12]:
srxToSeqS =libraryS.groupby(level=0).first()

In [13]:
validSrx=srxToSeqS.index.get_level_values(0)

## take only runs that satisfy both srx and srs filter

In [14]:
subAccessionDf=accessionDf[accessionDf.Sample.isin(uniqueSrsToSpeciesS.index)&accessionDf.Experiment.isin(srxToSeqS.index)]

In [15]:
subAccessionDf.loc[:,'ScientificName']=uniqueSrsToSpeciesS.loc[subAccessionDf.Sample.values].values[:]

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
subAccessionDf['Run']=subAccessionDf.index

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
publicRunDf=subAccessionDf

In [18]:
algnedSrx=srxToSeqS.loc[publicRunDf.Experiment].values
srrToSeqS=pd.Series(data=algnedSrx,index=publicRunDf.index)
publicRunDf.loc[:,'LibraryStrategy']=srrToSeqS[:]

In [19]:
publicRunDf['LibraryLayout']=layoutS.loc[publicRunDf['Experiment'].values].values

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
projectSubDf=projAccessDf.loc[publicRunDf.index]
projectSubDf.columns='proj_accession_'+projectSubDf.columns

In [21]:
yearAnnotatedPublicDf=pd.concat( [publicRunDf,projectSubDf],axis=1)#.iloc[8]

In [22]:
yearAnnotatedPublicDf['ScientificName']=yearAnnotatedPublicDf['ScientificName'].str.replace(' ','_')

In [23]:
import string
printable = set(string.printable)

removeNonAscii=lambda s: filter(lambda x: x in printable, s)
#removeNonAsciiCol=lambda col:col.apply(removeNonAscii)

In [24]:
subExportDf=yearAnnotatedPublicDf.loc[:,yearAnnotatedPublicDf.columns!='Run']


In [25]:
#### convert the micrbiome to something else

In [39]:
subExportDf['new_ScientificName']=subExportDf['ScientificName']

In [59]:
m_4=subExportDf['new_ScientificName'].str.contains('human')
#Homo_sapiens
subExportDf.loc[m_4, 'new_ScientificName']='Homo_sapiens'
#m_4=subExportDf['ScientificName'].str.contains('metagenome')
m_5=subExportDf['new_ScientificName'].str.contains('mouse')
#Homo_sapiens
subExportDf.loc[m_5, 'new_ScientificName']='Mus_musculus'
###for soil, to eliminate human contamination, align to human reference
m_6=subExportDf['new_ScientificName'].str.contains('^soil_metagenome#')
subExportDf.loc[m_6, 'new_ScientificName']='Homo_sapiens'


In [ ]:
for myCol in [u'ScientificName', u'LibraryStrategy',
       u'LibraryLayout','new_ScientificName']:
    subExportDf[myCol]=subExportDf[myCol].apply(removeNonAscii)

In [ ]:
subExportDf.to_csv(sra_dump_csv_dir,compression='gzip')

### create smaller version for pickle

In [ ]:
pickleExportInDf=subExportDf.copy()

In [ ]:
timeCols=['proj_accession_Updated'
          ,'proj_accession_Published'
          ,'proj_accession_Received']
for timeCol in timeCols:
    tmp_dateS=pickleExportInDf[timeCol].str.split('T').str[0]
    pickleExportInDf[timeCol]=pd.to_datetime( tmp_dateS)

In [ ]:
ignoreCols=['proj_accession_BioProject','proj_accession_Study','proj_accession_Spots','proj_accession_BioSample'
           ,'proj_accession_Sample','proj_accession_Experiment','proj_accession_Alias','proj_accession_Md5sum',
            'BioSample','proj_accession_Bases']

In [ ]:
subsetPickleDf=pickleExportInDf.loc[:,~pickleExportInDf.columns.isin(ignoreCols)]

In [ ]:
intTypes=['Spots','Bases']
for intType in intTypes:
    subsetPickleDf[intType]=pd.to_numeric(subsetPickleDf[intType],errors='coerce')

In [ ]:
factorTypes=[u'proj_accession_ReplacedBy', u'proj_accession_Type',
       u'proj_accession_Status', u'Status', u'proj_accession_Visibility',
       u'LibraryLayout', u'proj_accession_Loaded', u'LibraryStrategy',
       u'proj_accession_Updated', u'proj_accession_Received',
       u'proj_accession_Published', u'Member_Name', u'proj_accession_Center',
       u'ScientificName','new_ScientificName' ]#u'Study', u'proj_accession_Submission'

In [ ]:
for factorType in factorTypes:
    subsetPickleDf[factorType]=subsetPickleDf[factorType].astype('category')

In [ ]:
#sra_dump_csv_dir,sra_dump_pickle_dir
#subsetPickleDf.to_pickle(sra_dump_pickle_dir)

### spit in the extracted ids

In [53]:
import pandas as pd
import numpy as np
subsetPickleDf=pd.read_pickle(sra_dump_pickle_dir)

In [54]:
#subsetPickleDf.dtypes

In [55]:
#['Run_db','Run_digits']
subsetPickleDf['Run_db']=subsetPickleDf.index.str.extract('([EDS]RR)').values
subsetPickleDf['Run_digits']=subsetPickleDf.index.str.extract('[EDS]RR(\d+)').astype(np.int)

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
#subsetPickleDf.dtypes

In [57]:
subsetPickleDf.to_pickle(sra_dump_pickle_dir)

### scratch

In [ ]:
new_size=249733376L
old_size=1035668000L
new_size/float(old_size)

In [ ]:
scentific_VC=subExportDf['ScientificName'].value_counts()

In [43]:
#subExportDf

In [70]:
#scentific_VC.to_frame()

In [71]:
#scentific_VC[scentific_VC.index.str.contains('human')].iloc[:20]

In [72]:
#scentific_VC[scentific_VC.index.str.contains('mouse')].iloc[:20]